In [ ]:
#### demo_deviltask
#### simple implementation of the `devil' risk-taking task, based on Slovic (1966)
#### last updated 20181023 CRM

# experiment config

## how many trials?
ntrials = 5

## run on actual device?
# dev_mode runs a bit slower than interfacing with actual device, esp. makes a difference when colors cycling
use_device = False


#### program code ####
from ElGateau import ElGateau, Icon
%matplotlib qt5
ElGateau = ElGateau(dev_mode=(not use_device))
ElGateau.display_clear('all')

# config
bad_thing = 'devil'  # ('devil' or 'bomb')

# import experiment specific functions
import random

# load icons and reset
# unicode characters here work on Mac, but not Windows for some reason
icon_list = {'devil': '\U0001f47f', 'bomb': '\U0001f4a3',
             'box': '\U0000229e','bank': '\U0001f3e6',
             'money_single': '\U0001f4b5', 'money_accum': '\U0001f4b0',
             'clock': '\U0001f553'}
icon_size = {'devil': 50, 'bomb': 50,
             'box': 40,'bank': 30, 'money_single': 40, 'money_accum': 30, 'clock': 30}
icon = {}
for i in ('devil','bomb','box','bank','money_single'):
    icon[i] = Icon.text(icon_list[i],font='emoji',size=icon_size[i])
    
# do the point balance and clock icons separately, since need to shift the alignment
for i in ('money_accum','clock'):
    icon[i] = Icon.text(icon_list[i],position=(31,22),font='emoji',size=icon_size[i])

    
# experiment trials

# draw bank
ElGateau.display_update(1,icon['bank'])

# setup first trial
trial = 0
points = 0

# setup current trial
while trial < ntrials:
    ElGateau.display_update(2,Icon.text(str(trial+1),position=(31,48),ico=icon['clock'],size=20,font='LiberationMono-Bold'))
    ElGateau.display_update(5,Icon.text(str(points),position=(31,48),ico=icon['money_accum'],size=20,font='LiberationMono-Bold'))
    for button in range(1,11):
        ElGateau.display_update(button+5,icon['box'])

    locate_devil = list(range(1,11))
    random.shuffle(locate_devil)
    locate_devil = locate_devil[0]+5

    points_trial = 0
    box_next = 1
    trial_done = False
    while not trial_done:
        # push another image draw
        ElGateau.display_update(1,icon['bank'])

        button, rt = ElGateau.button_listen_key((1,box_next+5)) # either home or next box

        if button == 1:
            # keep your points and advance to next trial
            points = points + points_trial
            trial += 1
            trial_done = True
            
        elif button == (box_next+5):
            if button == locate_devil:
                # oops, got the devil! --and advance to next trial
                ElGateau.display_update(button,icon[bad_thing])
                # push another image draw
                ElGateau.display_update(1,icon['bank'])
                
                # implement feedbach
                points_trial = 0
                trial += 1
                trial_done = True
                button, rt = ElGateau.button_listen_key(1) # await home
                
            else:
                # add to current trial count
                ElGateau.display_update(button,icon['money_single'])
                # push another image draw
                ElGateau.display_update(1,icon['bank'])
                
                # implement feedback
                points_trial += 1
                box_next += 1

# end screen
ElGateau.reset()
ElGateau.display_update(8, Icon.text('Great\nJob', size=20))
ElGateau.display_update(13,Icon.text(str(points),position=(31,48),ico=icon['money_accum'],size=20,font='LiberationMono-Bold'))